In [11]:
import numpy as np
from matplotlib import pyplot as plt

### Define Model

In [2]:
'''
Model class for the network model
Store the model, including layer(construct by node), activation function
'''
class Model():
    def __init__(self, func, d_func, layer_nums, input_list, lr_rate):
        self.act_func = func                    # Activation function
        self.d_act_func = d_func                # Diviation of activate function

        self.layer_nums = layer_nums.copy()     # List of the number of each layer
                                                # 0 => input number
                                                # middle => layer number
                                                # last => output number

        self.input_list = input_list.copy()     # Store the list for model input
        self.result = 0                         # Initial network result
        self.lr_rate = lr_rate                  # Learning rate
        self.layer_list = np.full(len(self.layer_nums), None)
        self.loss = 0

        # Creating the network
        # self.intput_nodes = np.full(self.layer_nums[0], 0.0)   # Initial input variable
        # print( "0-th layer")
        self.layer_list[0] = Layer(self.act_func, self.d_act_func, len(self.input_list), self.input_list, self.lr_rate, True)
        for i in range(1, len(self.layer_nums)):
    #       print(i, "-th layer")
            self.layer_list[i] = Layer(self.act_func, self.d_act_func, self.layer_nums[i], self.layer_list[i-1].get_node_list().copy(), self.lr_rate, False)
    


    '''
    Calculate the network by using input data
    '''
    def cal_network(self, input):
        self.layer_list[0].set_input(input.copy())
        for i, layer in enumerate(self.layer_list):
            layer.cal_output()

        return self.get_result()

    '''
    Set output errors, for the last layer only
    '''
    def set_output_error(self, error):
        self.layer_list[len(self.layer_nums)-1].set_output_error(error)
  
    '''
    Adjust nodes in network using backpropagation and ground truth
    '''
    def adjust_model(self, ground_truth):
        # print(self.result, ground_truth)
        error = self.result - ground_truth
        # print(error)
        self.loss = np.dot(error, error) / 2
        self.set_output_error(error)
        for i in range(len(self.layer_list)-1, -1, -1):
            # print("in ", i, "-th layer")
            self.layer_list[i].adjust_weight()

    '''
    Return network result
    '''
    def get_result(self):
        self.result = self.layer_list[len(self.layer_nums)-1].get_output()
        return self.result

    def get_loss(self):
        return self.loss

    def get_output_w(self):
        w = self.layer_list[len(self.layer_list)-1].get_output_w(0)
        return w

### Define Layer

In [131]:
#@title
'''
Layer class for the network model
Help model to handle neurons
'''
class Layer_vec():
    '''
    Initial layer
    @param func - activation function
    @param d_func - diviation of activation function
    @param node_num - number of nodes in this layer
    @param last_layer - last layer's node list
    @param is_first - whether this layer is the first layer
    '''
    def __init__(self, func, d_func, node_num, last_layer, is_first):
        # Activation Functions
        self.act_func = func                                      # Activation function
        self.d_act_func = d_func                                  # Diviation of activate function

        # Input definition
        if not is_first:
            self.i_num = last_layer.get_node_num()               # Number of input node
        else:
            self.i_num = len(last_layer)                         # Number of input node
        self.input_vec = np.full(self.i_num+1, 0.0)              # Initial input passed from
        self.input_vec[self.i_num] = 1
        self.neuron_num = node_num
        self.last_layer = last_layer

        # Calculation variables
        self.w = np.random.rand(self.i_num+1, node_num)      # Initial weight
        self.w = np.full((self.i_num+1, node_num), 0.5)              # Initial input passed from
#         print(self.w)
        self.bp_vec = np.full(self.neuron_num, 0.0)                   # Recieve value passed from postorier layer
        self.is_first = is_first                                  # Set to true if this node is at first layer
        self.weighted_input = np.full(self.i_num, 0.0)           # Initial weighted input, use to store the value after the weighted input are sum up
        self.result = np.full(self.neuron_num, 0.0)                   # Initial output result, equal to the value after subsituted weighted input into activation function
        self.lr_rate = 0.005                                      # Learning rate of the node
  
    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self, lr_rate):
        self.lr_rate = lr_rate
        # Calculate each weight for the specific previous node
        delta = self.bp_vec * self.d_act_func(self.weighted_input)    # Dimation of layer node
        delta_w = np.outer(self.input_vec, delta)
        if (not self.is_first):
            pass_v = np.dot(delta, self.w[0:len(self.w)-1, :].transpose())
            self.last_layer.pass_bp(pass_v[0:len(self.input_vec)-1])
        self.w = self.w - self.lr_rate * delta_w

    def forwrad_pass(self):
        if not self.is_first:
            self.extract_value()
        self.bp_vec = np.full(self.neuron_num, 0.0)      # Set bp value to zero, for later adjustment

        # print(self.w, self.input_value)
        self.weighted_input = np.dot(self.input_value, self.w)
        self.result = self.act_func(self.weighted_input)
        return self.result
    
    '''
    Pass backpropagation value back to previous layer
    '''
    def pass_bp(self, bp_value):
        self.bp_vec = bp_value.copy()
        
    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_value = x.copy()
        if self.is_first:
            self.input_value = np.append(self.input_value, 1)
            
    def extract_value(self):
        self.input_value = self.last_layer.get_output()
        self.input_value = np.append(self.input_value, 1)

    def get_node_num(self):
        return self.neuron_num

    def set_output_error(self, error):
        if self.neuron_num != len(error):
            print("Output layer and error doesn't match")
            return
        self.pass_bp(error)

    def get_output(self):
        return self.result
    def get_output_w(self, i):
        return self.node_list[i].w

In [132]:
##############################################################
#                     test LAYER block                       #
##############################################################

lr_rate = 0.5
test_l = Layer_vec(ReLU, d_ReLU, 2, [1,2,3], True)
test_l2 = Layer_vec(ReLU, d_ReLU, 1, test_l, False)

for i in range(20):
    print(i, "'s round !!!")
    test_l.set_input([0.2, 0.1, 0.2])
    # print("test w: ", test.w)
    test_l.forwrad_pass()
    # print("test output: ", test.get_output())
    test_l2.forwrad_pass()
    print("test2 output: ", test_l2.get_output())
    error = test_l2.get_output() - 0.7
    test_l2.set_output_error([error])
    test_l2.adjust_weight(lr_rate)
    test_l.adjust_weight(lr_rate)


0 's round !!!
test2 output:  [1.25]
1 's round !!!
test2 output:  [0.8375]
2 's round !!!
test2 output:  [0.734375]
3 's round !!!
test2 output:  [0.70859375]
4 's round !!!
test2 output:  [0.70214844]
5 's round !!!
test2 output:  [0.70053711]
6 's round !!!
test2 output:  [0.70013428]
7 's round !!!
test2 output:  [0.70003357]
8 's round !!!
test2 output:  [0.70000839]
9 's round !!!
test2 output:  [0.7000021]
10 's round !!!
test2 output:  [0.70000052]
11 's round !!!
test2 output:  [0.70000013]
12 's round !!!
test2 output:  [0.70000003]
13 's round !!!
test2 output:  [0.70000001]
14 's round !!!
test2 output:  [0.7]
15 's round !!!
test2 output:  [0.7]
16 's round !!!
test2 output:  [0.7]
17 's round !!!
test2 output:  [0.7]
18 's round !!!
test2 output:  [0.7]
19 's round !!!
test2 output:  [0.7]


In [3]:
#@title
'''
Layer class for the network model
Help model to handle neurons
'''
class Layer():
    '''
    Initial layer
    @param func - activation function
    @param d_func - diviation of activation function
    @param node_num - number of nodes in this layer
    @param last_layer - last layer's node list
    @param is_first - whether this layer is the first layer
    '''
    def __init__(self, func, d_func, node_num, last_layer, lr_rate, is_first):
        # Activation Functions
        self.act_func = func                            # Activation function
        self.d_act_func = d_func                        # Diviation of activate function

        self.last_layer = last_layer                    # Input Layer num
        self.node_num = node_num                        # Number of nodes in this layer
        self.is_first = is_first                        # Set to true if this node is at first layer
        self.lr_rate = lr_rate                          # Learning rate of the node
        self.node_result = np.full(self.node_num, 0.0)  # List of all the nodes' output
        self.node_list = np.full(self.node_num, None)   # List of all the nodes
    #     print("in layer, ", self.node_num, self.last_layer)
        for i in range(self.node_num):
            if isinstance(self.last_layer[0], list):          # If the input is first layer, last layer may be 2-D array, for different input
                self.node_list[i] = node(self.act_func, self.d_act_func, self.last_layer[i], self.lr_rate, is_first)       # act_func, d_act_func, input node, is first
            else:
                self.node_list[i] = node(self.act_func, self.d_act_func, self.last_layer, self.lr_rate, is_first)       # act_func, d_act_func, input node, is first

    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self):
        for i in range(self.node_num):
            self.node_list[i].adjust_weight()

    def cal_output(self):
        # print("In layer")
        for i, node in enumerate(self.node_list):
            self.node_result[i] = node.cal_output()
        # print(self.node_result)

    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_value = x.copy()
        for i, node in enumerate(self.node_list):
            if isinstance(self.input_value[0], list):          # If the input is first layer, last layer may be 2-D array, for different input
                node.set_input(self.input_value[i])
            else:
                node.set_input(self.input_value)

    def get_node_list(self):
        return self.node_list.copy()

    def set_output_error(self, error):
        if self.node_num != len(error):
            print("Output layer and error doesn't match")
            return
        else:
            for i, node in enumerate(self.node_list):
                node.add_bp(error[i])

    def get_output(self):
        return self.node_result.copy()
    def get_output_w(self, i):
        return self.node_list[i].w

### Define Node

In [4]:
#@title
'''
Node class for the network model
Suppose to record input number, out number, weight, and the activation function
'''
class node():
    def __init__(self, func, d_func, i_node, lr_rate, is_first):
        # Activation Functions
        self.act_func = func                          # Activation function
        self.d_act_func = d_func                      # Diviation of activate function

        # Input definition
        self.i_num = len(i_node)                      # Number of input node
        self.i_node = i_node.copy()                   # Input node list
        self.input_value = np.full(len(i_node)+1, 0.0)  # Initial input passed from
        self.input_value[len(i_node)] = 1

        # Calculation variables
    #     self.w = np.full(len(i_node)+1, 0.5)            # Initial weight
        self.w = np.random.random((len(i_node)+1))            # Initial weight
        self.bp_value = 0.0                           # Recieve value passed from next layer
        self.is_first = is_first                      # Set to true if this node is at first layer
        self.weighted_input = 0.0                     # Initial weighted input, use to store the value after the weighted input are sum up
        self.result = 0.0                             # Initial output result, equal to the value after subsituted weighted input into activation function
        self.lr_rate = lr_rate                        # Learning rate of the node
        # print(self.w, self.input_value)

    '''
    Calculate output when new input is send into the node
    Need to set bp_value to zero, for the adjustment will occur after the calcualtion
    '''
    def cal_output(self):
        if not self.is_first:
          self.extract_value()
        self.bp_value = 0                     # Set bp value to zero, for later adjustment

        # print(self.w, self.input_value)
        self.weighted_input = np.dot(self.w, self.input_value)
        self.result = self.act_func(self.weighted_input)
        return self.result

    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self):
        # Calculate each weight for the specific previous node
        for i in range(self.i_num + 1):
            tilda = self.bp_value * self.d_act_func(self.weighted_input)
            delta_w = tilda * self.input_value[i]
            if (not self.is_first) and (i != self.i_num):
                self.i_node[i].add_bp(tilda * self.w[i])
            self.w[i] -= self.lr_rate * delta_w

    '''
    Get previous nodes value
    '''
    def extract_value(self):
        # Extract result from previous output
        for i, node in enumerate(self.i_node):
            self.input_value[i] = node.get_output()

    '''
    Add backpropagation value to this node
    @param pv - passed value from posterior node 
    '''
    def add_bp(self, pv):
        self.bp_value += pv

    '''
    Get result value
    '''
    def get_output(self):
        return self.result

    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_value = x.copy()
        if self.is_first:
            self.input_value = np.append(self.input_value, 1)
    

### Activation functions

In [46]:
#@title
'''
Activation function for the network
'''
def test_act_func(x):
    return x*11

'''
ReLU
'''
def ReLU(x):
    x[x<=0] = 0
    return x.copy()

'''
Sigmoid
'''
def Sigmoid(x):
    return 1/(1+np.exp(-x))


### Diviation of Activation function

In [47]:
#@title
'''
Diviation of the activation function for the network
'''
def d_test_act_func(x):
    return x+2

'''
Diviation of ReLU
'''
def d_ReLU(x):
    x[x > 0] = 1
    x[x < 0] = 0
    return x.copy()

'''
Diviation of Sigmoid
'''
def d_Sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s * (1 - s)

### Test Node class function


In [10]:
#############################################################
#                     test NODE block                       #
#############################################################
lr_rate = 0.5
test= node(ReLU, d_ReLU, [1,2,3], lr_rate, True)       # act_func, d_act_func, input node, is first
test2 = node(ReLU, d_ReLU, [test], lr_rate, False)

for i in range(20):
    print(i, "'s round !!!")
    test.set_input([0.2, 0.1, 0.2])
    # print("test w: ", test.w)
    test.cal_output()
    # print("test output: ", test.get_output())
    test2.cal_output()
    print("test2 output: ", test2.get_output())
    error = test2.get_output() - 0.7
    test2.add_bp(error)
    test2.adjust_weight()
    test.adjust_weight()

    # print("test w: ", test.w)
    # print("test2 w: ", test2.w)



0 's round !!!
test2 output:  0.6171759053310112
1 's round !!!
test2 output:  0.6975747801574688
2 's round !!!
test2 output:  0.6999955795656303
3 's round !!!
test2 output:  0.699999995696245
4 's round !!!
test2 output:  0.6999999999958164
5 's round !!!
test2 output:  0.6999999999999958
6 's round !!!
test2 output:  0.7
7 's round !!!
test2 output:  0.7
8 's round !!!
test2 output:  0.7
9 's round !!!
test2 output:  0.7
10 's round !!!
test2 output:  0.7
11 's round !!!
test2 output:  0.7
12 's round !!!
test2 output:  0.7
13 's round !!!
test2 output:  0.7
14 's round !!!
test2 output:  0.7
15 's round !!!
test2 output:  0.7
16 's round !!!
test2 output:  0.7
17 's round !!!
test2 output:  0.7
18 's round !!!
test2 output:  0.7
19 's round !!!
test2 output:  0.7


### Test Layer


In [45]:
##############################################################
#                     test LAYER block                       #
##############################################################

lr_rate = 0.5
test_l = Layer(ReLU, d_ReLU, 2, [1,2,3], lr_rate, True)
test_l2 = Layer(ReLU, d_ReLU, 1, test_l.get_node_list().copy(), lr_rate, False)

for i in range(20):
    print(i, "'s round !!!")
    test_l.set_input([0.2, 0.1, 0.2])
    # print("test w: ", test.w)
    test_l.cal_output()
    # print("test output: ", test.get_output())
    test_l2.cal_output()
    print("test2 output: ", test_l2.get_output())
    error = test_l2.get_output() - 0.7
    test_l2.set_output_error([error])
    test_l2.adjust_weight()
    test_l.adjust_weight()


0 's round !!!
test2 output:  [1.61324736]
1 's round !!!
test2 output:  [0.06102958]
2 's round !!!
test2 output:  [0.89549628]
3 's round !!!
test2 output:  [0.64329306]
4 's round !!!
test2 output:  [0.713997]
5 's round !!!
test2 output:  [0.69642701]
6 's round !!!
test2 output:  [0.70090361]
7 's round !!!
test2 output:  [0.69977095]
8 's round !!!
test2 output:  [0.70005803]
9 's round !!!
test2 output:  [0.6999853]
10 's round !!!
test2 output:  [0.70000373]
11 's round !!!
test2 output:  [0.69999906]
12 's round !!!
test2 output:  [0.70000024]
13 's round !!!
test2 output:  [0.69999994]
14 's round !!!
test2 output:  [0.70000002]
15 's round !!!
test2 output:  [0.7]
16 's round !!!
test2 output:  [0.7]
17 's round !!!
test2 output:  [0.7]
18 's round !!!
test2 output:  [0.7]
19 's round !!!
test2 output:  [0.7]


### Test Model

In [9]:
##############################################################
#                     test MODEL block                       #
##############################################################

lr_rate = 0.5
layer_nums = [2, 1]
layer_input = [[1, 1], [1], [1]]
test_m = Model(ReLU, d_ReLU, layer_nums, layer_input, lr_rate)
input_data = [[0.2, 0.1], [0.1], [0.2]]
ground_truth = 0.7
for i in range(20):
    print(i, "'s round !!!")
    test_m.cal_network(input_data)
    print("test_m output: ", test_m.get_result())
    test_m.adjust_model(ground_truth)
test_m.get_loss()
test_m.get_output_w()

0 's round !!!
test_m output:  [1.76569813]
1 's round !!!
test_m output:  [0.]
2 's round !!!
test_m output:  [0.]
3 's round !!!
test_m output:  [0.]
4 's round !!!
test_m output:  [0.]
5 's round !!!
test_m output:  [0.]
6 's round !!!
test_m output:  [0.]
7 's round !!!
test_m output:  [0.]
8 's round !!!
test_m output:  [0.]
9 's round !!!
test_m output:  [0.]
10 's round !!!
test_m output:  [0.]
11 's round !!!
test_m output:  [0.]
12 's round !!!
test_m output:  [0.]
13 's round !!!
test_m output:  [0.]
14 's round !!!
test_m output:  [0.]
15 's round !!!
test_m output:  [0.]
16 's round !!!
test_m output:  [0.]
17 's round !!!
test_m output:  [0.]
18 's round !!!
test_m output:  [0.]
19 's round !!!
test_m output:  [0.]


array([ 0.20920424,  0.5312474 ,  0.07663143, -0.23165184])

In [120]:
a = np.array([1, 2, 3])
b = np.array([2,1])
c = np.outer(b,a)
print(c)
print(np.dot(c, a))
print(np.zeros([2,3]))
a[a>2] = 100
print(a*a)


[[2 4 6]
 [1 2 3]]
[28 14]
[[0. 0. 0.]
 [0. 0. 0.]]
[    1     4 10000]
